# 0. Extract from MXL

In [71]:
import zipfile
import glob

filename = "XMLs/Alta_cancion.mxl"
with zipfile.ZipFile(filename, 'r') as zipfile:
    zipfile.extractall('XMLs/Alta_cancion')

# 1. Read XML

In [78]:
import xml.etree.cElementTree as ET

filename = "XMLs/至少還有你_主旋律.musicxml"
tree = ET.parse(filename)

In [79]:
root = tree.getroot()
root.tag, root.attrib

('score-partwise', {'version': '3.1'})

In [80]:
all_notes = root.findall('.//note')

In [81]:
lyric_notes = []
for note in all_notes:
    n = {}
    if note.find('lyric') is not None:
        n['type'] = note.find('type').text
        n['lyric'] = note.find('./lyric/text').text
        n['pitch'] = note.find('./pitch/step').text + note.find('./pitch/octave').text
        if note.find('./pitch/alter') is not None:
            if note.find('./pitch/alter').text == '-1':
                p = n['pitch'][0] + 'b' + n['pitch'][1]
                n['pitch'] = p
            else:
                n['pitch'] = n['pitch'][0] + '#' + n['pitch'][1]
                
        lyric_notes.append(n)

In [82]:
len(lyric_notes)

159

In [83]:
lyric_notes

[{'type': '16th', 'lyric': '我', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '怕', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '來', 'pitch': 'F#4'},
 {'type': '16th', 'lyric': '不', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '幾', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '我', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '要', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '抱', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '著', 'pitch': 'A4'},
 {'type': '16th', 'lyric': '你', 'pitch': 'B4'},
 {'type': '16th', 'lyric': '直', 'pitch': 'G4'},
 {'type': '16th', 'lyric': '到', 'pitch': 'G4'},
 {'type': 'eighth', 'lyric': '感', 'pitch': 'F#4'},
 {'type': 'eighth', 'lyric': '覺', 'pitch': 'A4'},
 {'type': 'eighth', 'lyric': '你', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '的', 'pitch': 'E5'},
 {'type': '16th', 'lyric': '皺', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric': '紋', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '有', 'pitch': 'D5'},
 {'type': '16th', 'lyric': '了', 'pitch': 'E5'},
 {'type': 'eighth', 'lyric

# 2. Exploring 

In [88]:
time = root.find('.//time')

In [90]:
time.find(".//beats").text

'4'

In [92]:
time.find(".//beat-type").text

'4'

# 3. Helper classes /functions

## 3.1 Helper classes

In [119]:
from enum import Enum

class NoteTypes(Enum):
    '''Class defining types of notes'''
    whole = 1
    half = 2
    quarter = 4
    eight = 8
    sixteenth = 16
    thirtysecond = 32

In [329]:
class Note:
    '''Class representing a musical note'''
    notetype = None
    step = ''
    octave = 0
    so = None
    tie = None
    
    def __init__(self,notetype,step,octave,tie):
        self.notetype = notetype
        self.step = step
        self.octave = octave
        self.tie = tie
        self.so = step + str(octave)
    
    def printNote(self):
        print('Note: ' + str(self.so) + ' / ' +
              str(self.notetype) + ' / ' +
              str(self.tie))

In [385]:
class Measure:
    '''Class representing a measure'''
    notes = []
    
    def __init__(self,notes):
        self.notes = notes
    
    def get_sos(self):
        sos = []
        for note in self.notes:
            sos.append(note.so)
        return sos
    
    def get_notetypes(self):
        notetypes = []
        for note in self.notes:
            notetypes.append(note.notetype)
        return notetypes 
    
    def printMeasure(self):
        for note in self.notes:
            note.printNote()

## 3.2 Helper functions

In [355]:
def get_ts_xml(root):
    '''Gets time signature for a musicxml file.
    Input: Root of XML Tree
    Output:top and bottom numbers of the Time signature'''
    time = root.find('.//time')
    top = int(time.find(".//beats").text)
    bottom = int(time.find(".//beat-type").text)
    return top,bottom

In [356]:
get_ts_xml(root)

(4, 4)

In [357]:
def get_bpm_xml(root):
    '''Gets beats per minute as well as type of beats for a musicxml file.
    Input: Root of XML Tree
    Output: beats per minute, noteType'''
    metronome = root.find('.//metronome')
    beats_per_min = int(metronome.find('.//per-minute').text)
    beat_type = metronome.find('.//beat-unit').text
    #beat_type = getattr(NoteTypes, str(beat_type))
    return beats_per_min, beat_type

In [358]:
get_bpm_xml(root)

(62, 'quarter')

In [376]:
def decode_xml_note(xml_note):
    '''Gets note type, step, octave and tie type'''
    try:
        notetype = xml_note.find('.//type').text
        if notetype == '16th':
            notetype = 'sixteenth'
        elif notetype == '32nd':
            notetype = 'thirtysecond'
    except:
        notetype = None
    try:
        step = xml_note.find('.//step').text
    except:
        step = ''
    try:
        octave = xml_note.find('.//octave').text
    except:
        octave = 0
    try:
        tie = xml_note.find('.//tie').text
    except:
        tie = None
        
    note = Note(notetype,step,octave,tie)
    #DEBUG
    #note.printNote()
    
    return note

In [361]:
measures = root.findall('.//measure')
xml_measure_notes = measures[4].findall('.//note')
xml_note = xml_measure_notes[0]
decode_xml_note(xml_note)

Note: F4 / eighth / None


In [377]:
# return all notes in a measure
def get_meas_notes_xml(xml_measure_notes):
    notes = []
    for xml_note in xml_measure_notes:
        note = decode_xml_note(xml_note)
        notes.append(note)
        #DEBUG
        #note.printNote()
    return notes

In [374]:
get_meas_notes_xml(xml_measure_notes)

Note: F4 / eighth / None
Note: F4 / eighth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None
Note: D5 / sixteenth / None
Note: D5 / sixteenth / None
Note: D5 / half / None
Note: D5 / half / None
Note: D5 / eighth / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None


## 3.3 Try to use classes

In [394]:
#get a note
xml_measures = root.findall('.//measure')
xml_measure_4 = xml_measures[4].findall('.//note')
xml_note = xml_measure_4[0]
note = decode_xml_note(xml_note)
note.printNote()

Note: F4 / eighth / None


In [382]:
note.so

'F4'

In [384]:
note.notetype

'eighth'

In [395]:
#get a measure
measure = Measure(get_meas_notes_xml(xml_measure_4))
measure.printMeasure()

Note: F4 / eighth / None
Note: A4 / sixteenth / None
Note: D5 / sixteenth / None
Note: D5 / half / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None


In [397]:
measure.get_sos()

['F4', 'A4', 'D5', 'D5', 'D5', 'A4', 'A4']

In [398]:
measure.get_notetypes()

['eighth',
 'sixteenth',
 'sixteenth',
 'half',
 'eighth',
 'sixteenth',
 'sixteenth']

# 4. Read XML and get measures to invert chords

In [399]:
xml_measures = root.findall('.//measure')
measures = []

for xml_measure in xml_measures:
    measure = Measure(get_meas_notes_xml(xml_measure))
    measures.append(measure)
    

In [400]:
for measure in measures:
    measure.printMeasure()

Note: 0 / None / None
Note: 0 / None / None
Note: 0 / None / None
Note: 0 / whole / None
Note: 0 / whole / None
Note: 0 / whole / None
Note: 0 / None / None
Note: 0 / half / None
Note: 0 / quarter / None
Note: 0 / eighth / None
Note: G4 / sixteenth / None
Note: G4 / sixteenth / None
Note: F4 / eighth / None
Note: A4 / sixteenth / None
Note: D5 / sixteenth / None
Note: D5 / half / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: A4 / sixteenth / None
Note: D5 / eighth / None
Note: A4 / sixteenth / None
Note: B4 / sixteenth / None
Note: B4 / half / None
Note: B4 / eighth / None
Note: G4 / sixteenth / None
Note: G4 / sixteenth / None
Note: 0 / None / None
Note: F4 / eighth / None
Note: A4 / eighth / None
Note: D5 / eighth / None
Note: E5 / sixteenth / None
Note: E5 / sixteenth / None
Note: E5 / sixteenth / None
Note: D5 / eighth / None
Note: 0 / eighth / None
Note: D5 / sixteenth / None
Note: E5 / sixteenth / None
Note: F5 / eighth / None
Note: C5 / sixteenth / None
Note: C